## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
### Camera calibration

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.misc import imsave
from scipy.misc import imresize
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

In [ ]:
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

# Undistort an example calibration image
img = mpimg.imread('camera_cal/calibration1.jpg')
dst = cv2.undistort(img, mtx, dist, None, mtx)

# Show distorted and undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Distorted', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted', fontsize=30)
extent = f.get_window_extent().transformed(f.dpi_scale_trans.inverted())
plt.savefig('output_images/calibration.jpg', bbox_inches=extent.expanded(1, 1.2), dpi=50)

### Distortion correction

In [ ]:
def undistort(img,mtx,dist,generate_examples=False):
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Save sample undistorted image
    
    if generate_examples:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,7))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title('Original distorted image', fontsize=30)
        ax2.imshow(undistorted)
        ax2.set_title('Undistorted image', fontsize=30)
        # Save figure
        extent = f.get_window_extent().transformed(f.dpi_scale_trans.inverted())
        plt.savefig('output_images/distortion.jpg', bbox_inches=extent.expanded(1, 1.2), dpi=50)
    
    return undistorted

### Thresholded binary image 

In [ ]:
def thresh_image(undistorted, sobel_kernel=3, h_thresh=(255,255), l_thresh=(255,255), s_thresh=(255, 255), sobelx_thresh=(255, 255), sobely_thresh=(255, 255), mag_thresh=(255,255), dir_thresh=(0,0), generate_examples=False):
    # HLS color space
    hls = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel > h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel > l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Sobel
    gray = cv2.cvtColor(undistorted, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sobelx_binary = np.zeros_like(scaled_sobelx)
    sobelx_binary[(scaled_sobelx >= sobelx_thresh[0]) & (scaled_sobelx <= sobelx_thresh[1])] = 1
    
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel) # Take the derivative in y
    abs_sobely = np.absolute(sobely) # Absolute y derivative to accentuate lines away from vertical
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    sobely_binary = np.zeros_like(scaled_sobely)
    sobely_binary[(scaled_sobely >= sobely_thresh[0]) & (scaled_sobely <= sobely_thresh[1])] = 1
    
    # Magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    # Direction
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1
    
    # Combine the binary thresholds
    # HLS (use only a combination of L and S that can handle shadows)
    hls_binary = np.zeros_like(s_binary)
    hls_binary[((l_binary == 0) & (s_binary == 1))] = 1
    
    # Sobel (x and y)
    sobel_binary = np.zeros_like(sobelx_binary)
    sobel_binary[(sobelx_binary == 1) & (sobely_binary == 1)] = 1
    
    # Magnitude and direction
    magdir_binary = np.zeros_like(mag_binary)
    magdir_binary[(mag_binary == 1) & (dir_binary == 1)] = 1
    
    # Combine all: pass if either HLS, Sobel or Magnitude is large enough
    combined_binary = np.zeros_like(hls_binary)
    combined_binary[(hls_binary == 1) | (sobel_binary == 1) | (magdir_binary == 1)] = 1
    
    # Show example
    if generate_examples:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))
        f.tight_layout()
        ax1.imshow(undistorted)
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(combined_binary, cmap='gray')
        ax2.set_title('Thresholded S', fontsize=30)
        extent = f.get_window_extent().transformed(f.dpi_scale_trans.inverted())
        plt.savefig('output_images/threshold.jpg', bbox_inches=extent.expanded(1, 1.2), dpi=50)
        imsave('output_images/thresholded.jpg',combined_binary.astype(np.bool_))
    
    return combined_binary

### Perspective transform

In [ ]:
def warp_image(undistorted, generate_examples=False):
    # src points were found manually with an external image editor
    src = np.float32([[1110,720],[685,448],[598,448],[217,720]])
    dst = np.float32([[1000,730],[1000,0],[300,0],[300,730]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src) # Inverse perspective matrix
    img_size = (undistorted.shape[1],undistorted.shape[0])
    warped_img = cv2.warpPerspective(undistorted, M, img_size, flags=cv2.INTER_LINEAR)
    
    # Save sample perspective transformed image
    if generate_examples:
        undistortedSrcPts = undistorted.copy()
        cv2.polylines(undistortedSrcPts,[src.astype(np.int32).reshape((-1,1,2))],True,(255,0,0),3)
        warped_imgDstPts = warped_img.copy()
        cv2.polylines(warped_imgDstPts,[dst.astype(np.int32).reshape((-1,1,2))],True,(255,0,0),3)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,7))
        f.tight_layout()
        ax1.imshow(undistortedSrcPts)
        ax1.set_title('Undistorted image with src points', fontsize=30)
        ax2.imshow(warped_imgDstPts)
        ax2.set_title('Warped image with dst points', fontsize=30)
        extent = f.get_window_extent().transformed(f.dpi_scale_trans.inverted())
        plt.savefig('output_images/perspective.jpg', bbox_inches=extent.expanded(1, 1.2), dpi=50)
    
    return warped_img, Minv

### Detect lane pixels

In [ ]:
def lane_pixels(warped_binary, visualize=False):
    # The following is adapted from Udacity sample code
    # Construct histogram to find initial lane positions
    histogram = np.sum(warped_binary[int(warped_binary.shape[0]/2):,:], axis=0)
    
    # Find left and right line as peaks in histogram
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Sliding (vertical) window to "track" lane lines
    out_img = np.dstack((warped_binary.astype(np.uint8), warped_binary.astype(np.uint8), warped_binary.astype(np.uint8)))*255

    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = np.int(warped_binary.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_binary.shape[0] - (window+1)*window_height
        win_y_high = warped_binary.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if visualize:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 6))
        f.tight_layout()
        # Plot histogram
        ax1.plot(histogram)
        ax1.plot((leftx_base, leftx_base), (0, np.amax(histogram,axis=0)+20), 'r-')
        ax1.plot((rightx_base, rightx_base), (0, np.amax(histogram,axis=0)+20), 'b-')
        ax1.set_title('Initial Lane Position Histogram', fontsize=30)
        # Plot lane pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        ax2.imshow(out_img)
        ax2.set_title('Lane pixels and sliding windows', fontsize=30)
        # Save figure
        extent = f.get_window_extent().transformed(f.dpi_scale_trans.inverted())
        plt.savefig('output_images/lane_pixels.jpg', bbox_inches=extent.expanded(1, 1.2), dpi=50)
    
    return leftx, lefty, rightx, righty

### Find road boundary

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

# Fit polygon to line pixels
def fit_polygon(x,y,line):
    # Fit a second order polynomial to the line coordinates, x and y
    line.current_fit = np.polyfit(y, x, 2)
    ploty = np.linspace(0, 719, num=720)
    line.allx = line.current_fit[0]*ploty**2 + line.current_fit[1]*ploty + line.current_fit[2]
    line.ally = ploty

### Curvature and vehicle position

In [ ]:
def road_stats(left_line,right_line):
    # Conversion from pixel to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/680 # meters per pixel in x dimension
    
    # Calculate left and right curvature in meters
    y_eval = np.max(left_line.ally)
    left_line.radius_of_curvature = ((1 + (2*left_line.current_fit[0]*y_eval*ym_per_pix + left_line.current_fit[1])**2)**1.5) / np.absolute(2*left_line.current_fit[0])
    right_line.radius_of_curvature = ((1 + (2*right_line.current_fit[0]*y_eval*ym_per_pix + right_line.current_fit[1])**2)**1.5) / np.absolute(2*right_line.current_fit[0])
    # Calculate mean of the two curvatures
    curvature = (left_line.radius_of_curvature+right_line.radius_of_curvature)/2
    
    # Calculate vehicle position on the road wrt. the center
    left_xstart = np.polyval(left_line.current_fit,720)
    right_xstart = np.polyval(right_line.current_fit,720)
    center_lane = (right_xstart+left_xstart)/2
    center_car = 1280 / 2
    position = (center_lane-center_car)*xm_per_pix
    
    return curvature, position

### Draw lane and display vehicle status

In [ ]:
def draw_lane(shape, left_line, right_line):
    # Create an image to draw the lines on
    warp_zero = np.zeros(shape).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_line.allx, left_line.ally]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.allx, right_line.ally])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    return color_warp

def draw_status(img, left_line, right_line):
    # Get road status (curvature and position)
    curvature, position = road_stats(left_line, right_line)

    curvature_txt = 'Radius  of Curvature = {0:0.3f}m'.format(curvature)
    position_txt = 'Vehicle is {0:0.3f}m left of center'.format(position)

    # Print road status onto the image
    cv2.putText(img, curvature_txt, (22, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, position_txt, (22, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    return img

## Pipeline function (combine all the above)

In [ ]:
def process_image(img, generate_examples=False):
    ## Correct for distortion (using above camera calibration)
    undistorted = undistort(img,mtx,dist,generate_examples)
        
    # Compute binary gradient image
    combined_binary = thresh_image(undistorted, sobel_kernel=3, 
                                     h_thresh=(10, 100), l_thresh=(0, 60), s_thresh=(85, 255),
                                     sobelx_thresh=(35, 100), sobely_thresh=(30, 255), 
                                     mag_thresh=(30, 255), dir_thresh=(0.7, 1.3), generate_examples=generate_examples)
    
    ## Perspective transform
    warped_img,Minv = warp_image(undistorted, generate_examples=generate_examples)
    warped_binary,_ = warp_image(combined_binary.astype(np.float32))
    warped_binary = warped_binary.astype(np.bool_)
    
    # Detect lane pixels
    leftx, lefty, rightx, righty = lane_pixels(warped_binary, generate_examples)
        
    # Instantiate a class for each line (left and right)
    left_line = Line()
    right_line = Line()

    # Fit polygons
    fit_polygon(leftx,lefty,left_line)
    fit_polygon(rightx,righty,right_line)
    
    # Plot fitted polygons
    if generate_examples:
        warped_img_overlaid = warped_img.copy()
        warped_img_overlaid[lefty,leftx,:] = [255,0,0]
        warped_img_overlaid[righty,rightx,:] = [0,0,255]
        plt.figure()
        plt.imshow(warped_img_overlaid)
        left_fitx = left_line.current_fit[0]*left_line.ally**2 + left_line.current_fit[1]*left_line.ally + left_line.current_fit[2]
        right_fitx = right_line.current_fit[0]*right_line.ally**2 + right_line.current_fit[1]*right_line.ally + right_line.current_fit[2]
        plt.plot(left_fitx, left_line.ally, color='yellow')
        plt.plot(right_fitx, right_line.ally, color='yellow')
        plt.title('Lane polygons', fontsize=30)
        plt.savefig('output_images/polygons.jpg')

    # Draw lane, same size as warped image
    warped_lane = draw_lane(warped_binary.shape, left_line, right_line)

    # Unwarp the lane image using inverse perspective matrix (Minv)
    unwarped_lane = cv2.warpPerspective(warped_lane, Minv, (warped_img.shape[1], warped_img.shape[0])) 
    
    # Overlay the unwarped lane image onto the original image
    result = cv2.addWeighted(undistorted, 1, unwarped_lane, 0.3, 0)
    
    # Print road status onto image
    result_status_overlaid = draw_status(result, left_line, right_line)
    
    return result_status_overlaid

## Pipeline (test images)

In [ ]:
# Make a list of test images
images = glob.glob('test_images/*.jpg')

#images = images[2:3]
images = images[3:4]

for idx,fname in enumerate(images):
    generate_examples = True
    
    # Load image
    img = mpimg.imread(fname)

    # Run processing pipeline
    result = process_image(img, generate_examples)
    
    # Save result
    imsave('output_images/' + fname,result)
    
    # Plot
    plt.figure(figsize=(15,8))
    plt.imshow(result)

## Pipeline (video)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Open video and process each frame
output = 'project_video_result.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)